In [1]:
%matplotlib inline
from nltk.stem.wordnet import WordNetLemmatizer
import string
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import re
import os
import gensim
from gensim.models import word2vec
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import MeanShift, estimate_bandwidth
from nltk.corpus import gutenberg, stopwords
from collections import Counter
os.getcwd()
os.chdir('/Users/kylehoward/desktop/thinkful/thinkful-2') 
os.getcwd()

'/Users/kylehoward/Desktop/Thinkful/Thinkful-2'

# NLP for scientific articles
This project is aimes at clustering scientific articles.

## Why are we interested?
There is so much scientific information online and countless resources to gain insights into scientific problems that I wanted to create a way to organize information based topic. This will help take disjointed information on the same topic and build resource databases

## Why this dataset?
Science is one of the fields where I believe data science can dramatically increase productivity and discovery by consolidating information and making it easier to access. I also wanted to create a model that succeeds in clustering articles within the same field.

## What are we trying to achieve?

The goal is to create a filter for articles and/or scientific documentation that can be used to unify scientific information. Once this model is successful it would make sense to create sub categories within the parent categories and continue to do so until we can easily access specific topics.

In [2]:
# All articles are from https://www.sciencenews.org
# Load Test
f = open("Article_2.txt", "r")
f = f.read()
#print(f)

In [3]:
# Getting 100 unique article names
text_list = []
for x in range(101):
    y = "Article_" + str(x) + ".txt"
    text_list.append(y)
# Adjust list because there is no article '0'
text_list = text_list[1:]
print(text_list)

['Article_1.txt', 'Article_2.txt', 'Article_3.txt', 'Article_4.txt', 'Article_5.txt', 'Article_6.txt', 'Article_7.txt', 'Article_8.txt', 'Article_9.txt', 'Article_10.txt', 'Article_11.txt', 'Article_12.txt', 'Article_13.txt', 'Article_14.txt', 'Article_15.txt', 'Article_16.txt', 'Article_17.txt', 'Article_18.txt', 'Article_19.txt', 'Article_20.txt', 'Article_21.txt', 'Article_22.txt', 'Article_23.txt', 'Article_24.txt', 'Article_25.txt', 'Article_26.txt', 'Article_27.txt', 'Article_28.txt', 'Article_29.txt', 'Article_30.txt', 'Article_31.txt', 'Article_32.txt', 'Article_33.txt', 'Article_34.txt', 'Article_35.txt', 'Article_36.txt', 'Article_37.txt', 'Article_38.txt', 'Article_39.txt', 'Article_40.txt', 'Article_41.txt', 'Article_42.txt', 'Article_43.txt', 'Article_44.txt', 'Article_45.txt', 'Article_46.txt', 'Article_47.txt', 'Article_48.txt', 'Article_49.txt', 'Article_50.txt', 'Article_51.txt', 'Article_52.txt', 'Article_53.txt', 'Article_54.txt', 'Article_55.txt', 'Article_56.txt', 

In [4]:
# Reading in all articles
article_list = []
for x in text_list:
    f = open(x, "r")
    f = f.read()
    article_list.append(f)
#print(article_list[:10])

In [5]:
# Cleaning data. Removing all characters except for letters. Removing whitespace 
w2v = ""
for art in article_list:
    work = art
    w2v = w2v + work


w2v = re.sub("[^a-zA-Z]"," ", w2v)
w2v = ' '.join(w2v.split())
len(w2v)

356922

In [6]:
# Tokenizing 
nlp = spacy.load('en')
w2v_tok = nlp(w2v)
#print(w2v_tok)

In [7]:
# Converting tokens to sentences and lemmas. Assuring punctuation is scrubbed
sentences = []
for sentence in w2v_tok.sents:
    sentence = [
        token.lemma_.lower()
        for token in sentence
        if not token.is_stop
        and not token.is_punct
    ]
    sentences.append(sentence)


#print(sentences)
print('We have {} tokens.'.format(len(w2v)))

We have 356922 tokens.


In [8]:
#------------------------------------------CBOW word2vec-----------------------------------------------
# Attempting to group words together based on proximity. Using CBOW due to smaller corpus
cbow_model = word2vec.Word2Vec(
    sentences,     
    min_count=5,   # Minimum word count threshold.
    window=15,     # Number of words around target word to consider.
    sg=0,          # Use CBOW because our corpus is small.
    sample=1e-3 ,  # Penalize frequent words.
    size=300,      # Word vector length.
    hs=1           # Use hierarchical softmax.
)


In [9]:
# word2vec analysis 
vocab = cbow_model.wv.vocab.keys()

# Testing to see which words it will group with 'planet'
print('Similarity to planet')
print(cbow_model.wv.most_similar('planet'))

# Testing for similarity 
print('Cosine similatrity analysis:')
print('study / research')
print(cbow_model.wv.similarity('study', 'research'))
print('atom / molecule')
print(cbow_model.wv.similarity('atom', 'molecule'))

# Testing to see if it will leave out the right word (looking for 'dinosaur')
print('Which doesnt fit?')
print('bat / bird / dinosaur / bee')
print(cbow_model.doesnt_match("bat bird dinosaur bee".split()))

'''function that adds up vectors, normalize vector?'''
# compare
# study / molecule
# atom / research
# unrelated
# more doesn't match
# dissimilar

Similarity to planet
[('dust', 0.9997695684432983), ('scientist', 0.9996869564056396), ('spacecraft', 0.9996809363365173), ('telescope', 0.9996238946914673), ('filament', 0.9995774626731873), ('phenomenon', 0.9995768070220947), ('size', 0.9995599985122681), ('difficult', 0.9995546936988831), ('universe', 0.9995542764663696), ('stream', 0.9995222091674805)]
Cosine similatrity analysis:
study / research
0.999560225565
atom / molecule
0.999296659344
Which doesnt fit?
bat / bird / dinosaur / bee
dinosaur


'function that adds up vectors, normalize vector?'

In [10]:
#-------------------------------------------tfidf-------------------------------------------------

X_train, X_test = train_test_split(article_list, test_size=0.4, random_state=0)

vectorizer = TfidfVectorizer(max_df=0.25,          # drop words that occur in more than a quarter of the articles
                             min_df=2,             # only use words that appear at least twice
                             stop_words='english', # picking list of stop words
                             lowercase=True,       # convert everything to lower case 
                             use_idf=True,         # use inverse document frequencies in our weighting
                             norm=u'l2',           # Treat longer and shorter paragraphs equally
                             smooth_idf=True       # Prevent divide-by-zero errors
                            )


# Applying the vectorizer
article_list_tfidf = vectorizer.fit_transform(article_list)
print("Number of features: %d" % article_list_tfidf.get_shape()[1])

X_train_tfidf, X_test_tfidf= train_test_split(article_list_tfidf, test_size=0.4, random_state=0)


# Reshape the vectorizer output to be legible 
X_train_tfidf_csr = X_train_tfidf.tocsr()

# number of articles
n = X_train_tfidf_csr.shape[0]

# Making a list of dictionaries, one per article
tfidf_bypara = [{} for _ in range(0,n)]

# List of features
terms = vectorizer.get_feature_names()

# for each article, lists the feature words and their tf-idf scores
for i, j in zip(*X_train_tfidf_csr.nonzero()):
    tfidf_bypara[i][terms[j]] = X_train_tfidf_csr[i, j]


print('Original article:', X_train[5])
print('Tf_idf vector:', tfidf_bypara[5])

Number of features: 3383
Original article: ﻿The deep waters of the East Pacific hold an unprepossessing treasure trove: potato-sized lumps of rock that contain valuable metals such as manganese, cobalt and copper. Turns out, such “manganese nodules” are home to another kind of goody: a species of sponge never before seen, researchers report online September 24 in Systematics and Biodiversity. These newly discovered nodule-dwellers may help scientists monitor the impact of future deep-sea mining.
Little is known about life in the abyssal depths of the ocean, 4,000 to 6,000 meters down. But the prospect of mining in those depths is looming: For example, the United Nation’s International Seabed Authority has granted 16 exploration contracts for mining manganese nodules. To track how mining will affect deep-sea ecosystems over time, scientists are eager to establish a baseline of existing biodiversity in regions such as the Clarion-Clipperton Zone (CCZ), an area in the eastern Pacific Ocea

In [11]:
# Applying SVD to reduce term document matrix into lower dimensional space (from 3417 to 260)
svd= TruncatedSVD(260)
lsa = make_pipeline(svd, Normalizer(copy=False))
# Run SVD on the training data and projecting the training data.
X_train_lsa = lsa.fit_transform(X_train_tfidf)

# Checking variance explained
variance_explained=svd.explained_variance_ratio_
total_variance = variance_explained.sum()
print("Percent variance captured by all components:",total_variance*100) # Upon review seems like overfitting

# Looking at what sorts of paragraphs our solution considers similar, for the first five identified topics
# It seems like some similarities exist but needs refining
paras_by_component=pd.DataFrame(X_train_lsa,index=X_train)

#Commenting this out for easier viewing on github
'''for i in range(5):
    print('Component {}:'.format(i))
    print(paras_by_component.loc[:,i].sort_values(ascending=False)[0:10])'''

Percent variance captured by all components: 100.0


"for i in range(5):\n    print('Component {}:'.format(i))\n    print(paras_by_component.loc[:,i].sort_values(ascending=False)[0:10])"

In [12]:
# Viewing the vectorizer output
print(X_train_tfidf_csr)

  (0, 0)	0.0410816381907
  (0, 10)	0.0225543949874
  (0, 11)	0.0701455913849
  (0, 13)	0.021439218399
  (0, 28)	0.0247840006085
  (0, 42)	0.0242922113966
  (0, 47)	0.0333747145272
  (0, 54)	0.0410816381907
  (0, 57)	0.0292635099454
  (0, 58)	0.0225543949874
  (0, 84)	0.0364352272037
  (0, 110)	0.0364352272037
  (0, 133)	0.0264423764715
  (0, 211)	0.0264423764715
  (0, 218)	0.0410816381907
  (0, 231)	0.0333747145272
  (0, 266)	0.0364352272037
  (0, 278)	0.0410816381907
  (0, 282)	0.0292635099454
  (0, 285)	0.0333747145272
  (0, 290)	0.0347768513407
  (0, 299)	0.0333747145272
  (0, 315)	0.0964803851191
  (0, 350)	0.0333747145272
  (0, 356)	0.0284720644908
  :	:
  (59, 1561)	0.141547507116
  (59, 1624)	0.180814062888
  (59, 1735)	0.109082695871
  (59, 1829)	0.160363650358
  (59, 1926)	0.180814062888
  (59, 1982)	0.180814062888
  (59, 1984)	0.180814062888
  (59, 2023)	0.10691816667
  (59, 2129)	0.180814062888
  (59, 2217)	0.0943611879807
  (59, 2342)	0.169296987813
  (59, 2343)	0.099269454

In [13]:
#--------------------------------------------LDA for clustering------------------------------------------
# Attempting to group each article with Latent Dirichlet Allocation
# Cleaning data with a function
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    # Remove stopwords
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    # Remove punctuation
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    # Get lemmas for all words
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    # Remove numbers and outliers
    normalized_2 = re.sub("[^a-zA-Z]"," ", w2v)
    # Remove extra whitespace
    normalized_3 = ' '.join(normalized_2.split())
    return normalized
# Apply function
doc_clean = [clean(doc).split() for doc in article_list]    

In [14]:
dictionary = corpora.Dictionary(doc_clean)

NameError: name 'corpora' is not defined

In [ ]:
# Creating the doc term matrix to use as our input
doc_term_matrix = [dictionary.doc2bow(article_list) for doc in article_list]

In [ ]:
# Creating the word vectors
word_vectors = cbow_model.wv.syn0

In [ ]:
# Building the model
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, 
               num_topics=10,        # With 100 articles I'm thinking 10% is a good test run
               id2word = dictionary, 
               alpha=1,              # number of topics
               passes=250)           # Number of passes over the corpus

In [ ]:
# Finding percent certainty for every category within each article
org = []
def get_org():
    for d in article_list:
        bow = dictionary.doc2bow(d.split())
        t = ldamodel.get_document_topics(bow)
        org.append(t)
        print(org)
    return org


In [ ]:
# Creating a variable to iterate over
lst = get_org()   

In [ ]:
# Test
x = lst[0]
print(len(x))

In [ ]:
# Test
print(x[1])
y = x[1]
print(y[1])

In [ ]:
# Iterating over the list to select the category that has the highest level of certainty and returning it to a list
maxes = []
for f in lst:
    mx = 0
    leng = len(f)
    counter = 0
    for x in f:
        counter += 1
        y = x[1]
        if y > mx:
            mx = y
            mxx = x
        if counter == leng:
            maxes.append(mxx)
        
print(maxes)           

In [ ]:
# Creating a list of catrgories to match with the article list
category = []
for x in maxes:
    y = x[0]
    category.append(y)
print(category)
len(category)

In [ ]:
# Creating dataframe 
df = pd.DataFrame()


In [ ]:
# df continued
df['category'] = category
df['article'] = text_list

In [ ]:
# Checking our dataframe
df.head(10)

In [ ]:
# Export to compare to article topic list in excel
df.to_csv('lda_results')

In [ ]:
# Category 4 was 100% accurate while others need more manipulation